In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -qU transformers==4.56.1 datasets==3.2.0 optimum==1.24.0
# !pip install -qU json-repair==0.29.1
!pip install -qU vllm==0.7.2


!pip install -U transformers accelerate bitsandbytes peft

pip install --upgrade accelerate transformers
pip install --upgrade protobuf==3.20.*


In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

import wandb
wandb_key = user_secrets.get_secret("wandb")
hf_token = user_secrets.get_secret("huggingface")

wandb.login(key=wandb_key)
!huggingface-cli login --token $hf_token

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aahmedasd936 (aahmedasd936-helwan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Ahmed Salah44` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Ahmed Salah44`


In [ ]:
# import json
# import os
from os.path import join
# import random
# from tqdm.auto import tqdm
import requests
import pandas as pd


from pydantic import BaseModel, Field
# from typing import List, Optional, Literal
# from datetime import datetime

# import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

data_dir = "/gdrive/MyDrive/Fine tuning"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

# def parse_json(text):
#     try:
#         return json_repair.loads(text)
#     except:
#         return None

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful medical assistant. "
            "Answer questions clearly in English. "
            "If the question is dangerous (like medication dosage for children), reply with: 'Consult a qualified doctor.'"
        )
    },
    {
        "role": "user",
        "content": "What is the standard treatment for Erdheim–Chester disease?"
    }
]


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

2025-09-28 10:57:36.549670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759057056.574892     341 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759057056.582208     341 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
response

"The standard treatment for Erdheim-Chester disease involves chemotherapy, which can include drugs like doxorubicin and cyclophosphamide. Additionally, immunotherapy may be used to help control the immune system's response to the cancer cells. It's important to consult with a qualified oncologist or hematologist who specializes in this type of rare cancer for personalized advice based on individual patient circumstances."

In [ ]:
sft_data_path = join(data_dir, "datasets", "medquad.csv")
df=pd.read_csv(sft_data_path)
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
llm_finetunning_data = []

system_message = "\n".join([
    "You are a helpful medical assistant.",
    "Answer medical questions clearly and concisely in English.",
    "If the question is dangerous (like medication dosages for children), reply with: 'Consult a qualified doctor.'"
])


for _, row in df.iterrows():
    rec = {
        "system": system_message,
        "instruction": f"Question: {row['question']}",
        "input": "",
        "output": row['answer'],
        "history": [],
        # "metadata": {
        #     "source": row['source'],
        #     "focus_area": row['focus_area']
        # }
    }
    llm_finetunning_data.append(rec)





random.Random(101).shuffle(llm_finetunning_data)

In [ ]:
llm_finetunning_data

[{'system': "You are a helpful medical assistant.\nAnswer medical questions clearly and concisely in English.\nIf the question is dangerous (like medication dosages for children), reply with: 'Consult a qualified doctor.'",
  'instruction': 'Question: What are the treatments for progressive external ophthalmoplegia ?',
  'input': '',
  'output': 'These resources address the diagnosis or management of progressive external ophthalmoplegia:  - Gene Review: Gene Review: Mitochondrial DNA Deletion Syndromes  - Gene Review: Gene Review: POLG-Related Disorders  - Genetic Testing Registry: Autosomal dominant progressive external ophthalmoplegia with mitochondrial DNA deletions 1  - Genetic Testing Registry: Autosomal dominant progressive external ophthalmoplegia with mitochondrial DNA deletions 2  - Genetic Testing Registry: Autosomal dominant progressive external ophthalmoplegia with mitochondrial DNA deletions 3  - Genetic Testing Registry: Progressive external ophthalmoplegia  - United Mito

In [ ]:
len(llm_finetunning_data)

16412

to add it ro drive  (when using colab)

In [ ]:
train_sample_sz = 16000

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    # json.dump(train_ds, dest, ensure_ascii=False, default=str)
    for rec in train_ds:
        dest.write(json.dumps(rec, ensure_ascii=False, default=str) + "\n")

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    # json.dump(eval_ds, dest, ensure_ascii=False, default=str)
    for rec in eval_ds:
        dest.write(json.dumps(rec, ensure_ascii=False, default=str) + "\n")

download data from drive and add it manully to kaggle input

In [ ]:
import json

with open("/kaggle/working/LLaMA-Factory/data/dataset_info.json", "r", encoding="utf-8") as f:
    dataset_info = json.load(f)

dataset_info["medical_finetune_train"] = {
    "file_name": "/kaggle/input/train-json/train.json",
    "columns": {
        "prompt": "instruction",
        "query": "input",
        "response": "output",
        "system": "system",
        "history": "history"
    }
}

dataset_info["medical_finetune_val"] = {
    "file_name": "/kaggle/input/val-json/val (2).json",
    "columns": {
        "prompt": "instruction",
        "query": "input",
        "response": "output",
        "system": "system",
        "history": "history"
    }
}

with open("/kaggle/working/LLaMA-Factory/data/dataset_info.json", "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, indent=4, ensure_ascii=False)



In [ ]:
%%writefile /kaggle/working/LLaMA-Factory/examples/train_lora/medical_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: medical_finetune_train
eval_dataset: medical_finetune_val
template: qwen
cutoff_len: 3500
#----------------------------------------
# max_samples: 50
#----------------------------------------
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /kaggle/working/models/checkpoint-1500
output_dir: /kaggle/working/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
# ddp_timeout: 180000000
save_safetensors: true
# ddp_backend: none
# num_processes: 1

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: medical-finetune-llamafactory

# push_to_hub: true
# export_hub_model_id: "A7medsala7/medical-finetune"
# hub_private_repo: true
# hub_strategy: checkpoint


In [ ]:
import os
import json
from os.path import join

train_sample_sz = 16000

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

def force_stringify(dataset):
    fixed = []
    for row in dataset:
        row["output"] = str(row.get("output", ""))
        row["input"] = str(row.get("input", ""))
        row["instruction"] = str(row.get("instruction", ""))
        fixed.append(row)
    return fixed

train_ds = force_stringify(train_ds)
eval_ds = force_stringify(eval_ds)




to add it again to drive

In [ ]:
os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    # json.dump(train_ds, dest, ensure_ascii=False, default=str)
    for rec in train_ds:
        dest.write(json.dumps(rec, ensure_ascii=False, default=str) + "\n")

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    # json.dump(eval_ds, dest, ensure_ascii=False, default=str)
    for rec in eval_ds:
        dest.write(json.dumps(rec, ensure_ascii=False, default=str) + "\n")

In [ ]:
%%writefile /kaggle/working/LLaMA-Factory/examples/train_lora/medical_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: medical_finetune_train
eval_dataset: medical_finetune_val
template: qwen
cutoff_len: 3500
#----------------------------------------
# max_samples: 50
#----------------------------------------
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /kaggle/working/checkpoint-4500-clean/checkpoint-4500
output_dir: /kaggle/working/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
# ddp_timeout: 180000000
save_safetensors: true
# ddp_backend: none
# num_processes: 1

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 200

report_to: wandb
run_name: medical-finetune-llamafactory

# push_to_hub: true
# export_hub_model_id: "A7medsala7/medical-finetune"
# hub_private_repo: true
# hub_strategy: checkpoint


Writing /kaggle/working/LLaMA-Factory/examples/train_lora/medical_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /kaggle/working/LLaMA-Factory/examples/train_lora/medical_finetune.yaml 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-09-28 11:00:29.931139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759057229.953363     400 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759057229.960664     400 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 09-28 11:00:41 __init__.py:190] Automatically detected platform cuda.
[INFO|2025-09-28 11:00:45] llamafactory.cli:143 >> Initializing 2 distributed tasks at: 127.0.0.1:33269
W0928 11:00:47.810000 422 torch/distributed/run.py:793] 
W0928 11:00:47.810000 422 torch/distributed/run.py:793] *****************************************
W0928 11:00:47.810000 422 torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable fo

In [ ]:
!zip -r models.zip /kaggle/working/models


  adding: kaggle/working/models/ (stored 0%)
  adding: kaggle/working/models/checkpoint-5000/ (stored 0%)
  adding: kaggle/working/models/checkpoint-5000/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/models/checkpoint-5000/tokenizer.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 81%)
  adding: kaggle/working/models/checkpoint-5000/added_tokens.json (deflated 67%)
  adding: kaggle/working/models/checkpoint-5000/README.md (deflated 65%)
  adding: kaggle/working/models/checkpoint-5000/chat_template.jinja (deflated 71%)
  adding: kaggle/working/models/checkpoint-5000/scheduler.pt (deflated 56%)
  adding: kaggle/working/models/checkpoint-5000/vocab.json (deflated 61%)
  adding: kaggle/working/models/checkpoint-5000/trainer_state.json (deflated 82%)
  adding: kaggle/working/models/checkpoint-5000/rng_state_1.pth (deflated 26%)
  adding: kaggle/working/models/checkpoint-5000/merges.txt (deflated 57%)
  adding: kaggle/working/models/checkpoint-5000/special_tokens_map.json (deflated 69%)
  adding: kaggle/working/models/checkpoint-5000/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/models/checkpoint-5000/optimizer.pt (deflated 9%)
  adding: kaggle/working/models/checkpoint-5000/training_args.bin (deflated 51%)
  adding: kaggle/working/models

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
response

"The standard treatment for Erdheim-Chester disease involves chemotherapy, which can include drugs like doxorubicin and cyclophosphamide. Additionally, immunotherapy may be used to help control the immune system's response to the cancer cells. It's important to consult with a qualified oncologist or hematologist who specializes in this type of rare cancer for personalized advice based on individual patient circumstances."

In [ ]:
evaluation_questions = [
    "What is the standard treatment for Erdheim–Chester disease?",
    "What is the most common mutation associated with Erdheim–Chester disease?",
    "How does Erdheim–Chester disease differ from Langerhans cell histiocytosis in terms of cell involvement?",
    "What role does interferon-alpha play in the treatment of Erdheim–Chester disease?",
    "List some common cardiovascular or neurological complications of Erdheim–Chester disease.",
    "A 45-year-old patient presents with persistent bone pain, and X-rays show bilateral cortical sclerosis of long bones. What differential diagnoses should be considered?",
    "A patient with Erdheim–Chester disease is found to have a BRAF V600E mutation. What targeted therapy is appropriate?",
    "If no BRAF mutation is detected, what alternative targeted therapy could be considered?",
    "In what situations might radiation therapy be useful for Erdheim–Chester disease?",
    "How would you explain Erdheim–Chester disease in simple terms to a high school student?",
    "Is there a single globally standardized protocol for Erdheim–Chester disease approved by the FDA?",
    "Which targeted therapy for Erdheim–Chester disease received FDA approval in 2017?",
    "Can conventional chemotherapy (e.g., doxorubicin or cyclophosphamide) be considered a standard treatment for Erdheim–Chester disease today?",
    "Who won the last Liverpool football match?",
    "What is the current exchange rate of USD to Egyptian Pound today?",
    "What is the recipe for chocolate cake?"
]

responses = []

for q in evaluation_questions:
    # Prepare chat messages
    messages = [
        {"role": "user", "content": q}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Generate
    generated_ids = model.generate(
        model_inputs.input_ids,
     max_new_tokens=1024,
     do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    # Remove the prompt tokens
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode response
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append({"question": q, "answer_with_out_adapter": response})

# Show results
for r in responses:
    print("Q:", r["question"])
    print("A:", r["answer_with_out_adapter"])
    print("-" * 80)


Q: What is the standard treatment for Erdheim–Chester disease?
A: Erdheim-Chester病是一种罕见的慢性炎症性疾病，主要影响中枢神经系统。目前尚无根治方法，但有一些治疗方法可以缓解症状和延缓疾病进展。

1. **药物治疗**：常用的药物包括皮质类固醇（如泼尼松）和其他免疫抑制剂（如环磷酰胺、甲氨蝶呤等）。这些药物可以帮助控制炎症反应，减轻症状。

2. **手术治疗**：在某些情况下，如果出现严重的脑积水或其他并发症，可能需要进行外科手术来减轻压力或移除肿块。

3. **支持性治疗**：包括物理治疗、语言治疗等，以帮助患者维持最佳的生活质量。

4. **临床试验**：对于一些难治性的病例，参与临床试验可能是获得最新治疗方法的机会。

5. **生活方式调整**：保持健康的生活方式，如均衡饮食、适量运动、避免吸烟和过量饮酒，也有助于改善病情。

请注意，具体的治疗方案应由专业的医疗团队根据患者的实际情况制定。如果您或您认识的人患有此病，请务必咨询专业医生获取个性化建议。
--------------------------------------------------------------------------------
Q: What is the most common mutation associated with Erdheim–Chester disease?
A: Erdheim-Chester disease (EDC) is a rare and complex autoimmune disorder characterized by chronic inflammation of the lymph nodes, particularly in the head and neck regions. The exact cause of EDC remains unclear, but it is believed to involve both genetic predisposition and environmental factors.

The most common mutations associated with EDC include:

1.

In [ ]:
df = pd.DataFrame(responses)
df

,question,answer_with_out_adapter
0,What is the standard treatment for Erdheim–Che...,Erdheim-Chester病是一种罕见的慢性炎症性疾病，主要影响中枢神经系统。目前尚无根...
1,What is the most common mutation associated wi...,Erdheim-Chester disease (EDC) is a rare and co...
2,How does Erdheim–Chester disease differ from L...,Erdheim-Chester Disease and Langerhans Cell Hi...
3,What role does interferon-alpha play in the tr...,Interferon-α (IFN-α) is not typically used as ...
4,List some common cardiovascular or neurologica...,Erdheim-Chester disease is a rare and complex ...
5,A 45-year-old patient presents with persistent...,The symptoms described—persistent bone pain an...
6,A patient with Erdheim–Chester disease is foun...,The appropriate targeted therapy for a patient...
7,"If no BRAF mutation is detected, what alternat...",If a patient with melanoma does not have a BRA...
8,In what situations might radiation therapy be ...,Radiation therapy is not typically used as the...
9,How would you explain Erdheim–Chester disease ...,Erdheim-Chester disease is a rare and complex ...


In [ ]:
df.to_csv("model_evaluation_results.csv", index=False)

In [ ]:
finetuned_model_id = "/kaggle/working/models"
model.load_adapter(finetuned_model_id)

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
response

"How might Erdheim-Chester disease be treated? There is no cure for Erdheim-Chester disease, but there are treatments that can help to relieve symptoms and improve quality of life. The most effective therapy has not yet been determined.  Some people respond well to corticosteroids such as prednisone or methylprednisolone.  Immunosuppressive therapies may also be used alone or in combination with steroids.  These include azathioprine, cyclophosphamide, cyclosporine A, mycophenolate mofetil, tacrolimus, and infliximab.  Other immunomodulatory agents including interferon alpha-2b, rituximab, thalidomide, lenalidomide, and belimumab have also shown some promise.  Radiation therapy and surgery may be useful in certain cases.  More detailed information about the management of Erdheim-Chester disease is available on eMedicine's Web site and can be viewed by clicking here."

In [ ]:
evaluation_questions = [
    "What is the standard treatment for Erdheim–Chester disease?",
    "What is the most common mutation associated with Erdheim–Chester disease?",
    "How does Erdheim–Chester disease differ from Langerhans cell histiocytosis in terms of cell involvement?",
    "What role does interferon-alpha play in the treatment of Erdheim–Chester disease?",
    "List some common cardiovascular or neurological complications of Erdheim–Chester disease.",
    "A 45-year-old patient presents with persistent bone pain, and X-rays show bilateral cortical sclerosis of long bones. What differential diagnoses should be considered?",
    "A patient with Erdheim–Chester disease is found to have a BRAF V600E mutation. What targeted therapy is appropriate?",
    "If no BRAF mutation is detected, what alternative targeted therapy could be considered?",
    "In what situations might radiation therapy be useful for Erdheim–Chester disease?",
    "How would you explain Erdheim–Chester disease in simple terms to a high school student?",
    "Is there a single globally standardized protocol for Erdheim–Chester disease approved by the FDA?",
    "Which targeted therapy for Erdheim–Chester disease received FDA approval in 2017?",
    "Can conventional chemotherapy (e.g., doxorubicin or cyclophosphamide) be considered a standard treatment for Erdheim–Chester disease today?",
    "Who won the last Liverpool football match?",
    "What is the current exchange rate of USD to Egyptian Pound today?",
    "What is the recipe for chocolate cake?"
]

responses = []

for q in evaluation_questions:
    # Prepare chat messages
    messages = [
        {"role": "user", "content": q}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Generate
    generated_ids = model.generate(
        model_inputs.input_ids,
     max_new_tokens=1024,
     do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    # Remove the prompt tokens
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode response
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    responses.append({"question": q, "answer": response})

# Show results
for r in responses:
    print("Q:", r["question"])
    print("A:", r["answer"])
    print("-" * 80)


Q: What is the standard treatment for Erdheim–Chester disease?
A: How might Erdheim–Chester disease be treated? There is no cure for this condition and currently there is no specific therapy to treat it. Treatment is aimed at alleviating symptoms and improving quality of life. The choice of treatment depends on many factors including the severity of the condition, the signs and symptoms present in each person, and the response to previous treatments. In some cases, corticosteroids may be used alone or with other medications (e.g., cyclophosphamide) to reduce inflammation. Other therapies that have been tried include:  Interferon alpha  tumor necrosis factor inhibitors (anti-TNFs) such as infliximab, adalimumab, golimumab, etanercept, rituximab, anakinra, certolizumab, natalizumab, abatacept, and belimumab  interleukin-1 receptor antagonists (IL-1Ra) such as canakinumab, tocilizumab, sarilumab, and eculizumab  histone deacetylase inhibitors (HDACIs) such as panobinostat  immune checkpoi

In [ ]:
len(responses)

16

In [ ]:
len(evaluation_questions)

16

In [ ]:
df = pd.DataFrame(responses)
df

,question,answer
0,What is the standard treatment for Erdheim–Che...,How might Erdheim–Chester disease be treated? ...
1,What is the most common mutation associated wi...,The most common genetic cause of this conditio...
2,How does Erdheim–Chester disease differ from L...,The cells involved in the two conditions are d...
3,What role does interferon-alpha play in the tr...,How might interferon-alpha be used to treat Er...
4,List some common cardiovascular or neurologica...,Erdheim–Chester disease may cause several type...
5,A 45-year-old patient presents with persistent...,The following conditions may present with simi...
6,A patient with Erdheim–Chester disease is foun...,BRAF inhibitors are approved for the treatment...
7,"If no BRAF mutation is detected, what alternat...",Imatinib mesylate (Gleevec) is an oral tyrosin...
8,In what situations might radiation therapy be ...,Radiation therapy is used to treat bone lesion...
9,How would you explain Erdheim–Chester disease ...,Erdheim-Chester disease is an autoimmune disor...


In [ ]:
df.to_csv("model_evaluation_results.csv", index=False)

In [ ]:
import pandas as pd

# Load both CSV files
df_with_adapter = pd.read_csv("model_evaluation_results.csv")
df_without_adapter = pd.read_csv("model_evaluation_results_no_adapter.csv")


In [ ]:
# Merge on 'question'
merged_df = pd.merge(
    df_with_adapter,
    df_without_adapter,
    on="question",
    how="inner"
)

In [ ]:



# Save to new CSV
merged_df.to_csv("merged_model_evaluation_results.csv", index=False)

print("✅ Merged file saved as merged_model_evaluation_results.csv")


✅ Merged file saved as merged_model_evaluation_results.csv


In [ ]:
merged_df

,question,answer,answer_with_out_adapter
0,What is the standard treatment for Erdheim–Che...,How might Erdheim–Chester disease be treated? ...,Erdheim-Chester病是一种罕见的慢性炎症性疾病，主要影响中枢神经系统。目前尚无根...
1,What is the most common mutation associated wi...,The most common genetic cause of this conditio...,Erdheim-Chester disease (EDC) is a rare and co...
2,How does Erdheim–Chester disease differ from L...,The cells involved in the two conditions are d...,Erdheim-Chester Disease and Langerhans Cell Hi...
3,What role does interferon-alpha play in the tr...,How might interferon-alpha be used to treat Er...,Interferon-α (IFN-α) is not typically used as ...
4,List some common cardiovascular or neurologica...,Erdheim–Chester disease may cause several type...,Erdheim-Chester disease is a rare and complex ...
5,A 45-year-old patient presents with persistent...,The following conditions may present with simi...,The symptoms described—persistent bone pain an...
6,A patient with Erdheim–Chester disease is foun...,BRAF inhibitors are approved for the treatment...,The appropriate targeted therapy for a patient...
7,"If no BRAF mutation is detected, what alternat...",Imatinib mesylate (Gleevec) is an oral tyrosin...,If a patient with melanoma does not have a BRA...
8,In what situations might radiation therapy be ...,Radiation therapy is used to treat bone lesion...,Radiation therapy is not typically used as the...
9,How would you explain Erdheim–Chester disease ...,Erdheim-Chester disease is an autoimmune disor...,Erdheim-Chester disease is a rare and complex ...


### Cost Estimation

In [ ]:
def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response



In [ ]:
from tqdm.auto import tqdm
import random
from datetime import datetime

start_time = datetime.now()

input_tokens = 0
output_tokens = 0

for i in tqdm(evaluation_questions):

    messages = [
        {
            "role": "user",
            "content": i,
        }
    ]

    response = generate_resp(messages)

    input_tokens += len(tokenizer.apply_chat_template(messages))
    output_tokens += len(tokenizer.encode(response))
    print(i)
print("----"*40)
total_time = (datetime.now() - start_time).total_seconds()

print(f"Total Time: {total_time} seconds")
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {input_tokens + output_tokens}")

  0%|          | 0/16 [00:00<?, ?it/s]

What is the standard treatment for Erdheim–Chester disease?
What is the most common mutation associated with Erdheim–Chester disease?
How does Erdheim–Chester disease differ from Langerhans cell histiocytosis in terms of cell involvement?
What role does interferon-alpha play in the treatment of Erdheim–Chester disease?
List some common cardiovascular or neurological complications of Erdheim–Chester disease.
A 45-year-old patient presents with persistent bone pain, and X-rays show bilateral cortical sclerosis of long bones. What differential diagnoses should be considered?
A patient with Erdheim–Chester disease is found to have a BRAF V600E mutation. What targeted therapy is appropriate?
If no BRAF mutation is detected, what alternative targeted therapy could be considered?
In what situations might radiation therapy be useful for Erdheim–Chester disease?
How would you explain Erdheim–Chester disease in simple terms to a high school student?
Is there a single globally standardized protoc

In [ ]:
3852/221

17.429864253393664

## vLLM

In [ ]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_model_id = "/kaggle/working/models"


# !nohup vllm serve "{base_model_id}" --dtype=half --gpu-memory-utilization 0.8 --max_lora_rank 64 --enable-lora --lora-modules medical-lora="{adapter_model_id}" &


!vllm serve "Qwen/Qwen2.5-1.5B-Instruct" \
    --dtype=half \
    --gpu-memory-utilization 0.8 \   
    --max_lora_rank 64 \
    --enable-lora \
    --lora-modules medical-lora="/kaggle/working/models"


In [ ]:
!tail -n 30 nohup.out